In [ ]:
%load_ext autoreload
%autoreload 2

# Random Forest Algorithm

Random Forest is an ensemble learning method predominantly used for classification and regression tasks. It operates by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean/average prediction (regression) of the individual trees. Random Forests correct for decision trees' habit of overfitting to their training set.

## Key Concepts

- **Ensemble Learning**: Technique of combining the predictions from multiple machine learning algorithms to make more accurate predictions than any individual model.
- **Decision Tree**: A decision support tool that uses a tree-like model of decisions and their possible consequences. It's the building block of a Random Forest.

## How Random Forest Works

1. **Bootstrap Aggregating (Bagging)**: Random Forest uses the bagging model and aggregates predictions by voting for the most popular output class or averaging in the case of regression.
   
2. **Feature Randomness**: When growing each tree, each time a split is considered, a random sample of `m` features is chosen as split candidates from the full set of `p` features. The split only considers these `m` features rather than the best split among all features. This process is known as the random subspace method.
   
   - Typically, for a classification problem with `p` features, `sqrt(p)` (rounded down) features are used in each split.
   - For a regression problem, `p/3` features are used in each split, with a minimum node size of 5 as the default.

## Equations

- **Gini Impurity** (used to calculate the quality of a split):
  
  $$ Gini = 1 - \sum_{i=1}^{n} p_i^2 $$

  Where `p_i` is the ratio of class `i` instances among the training instances in the dataset.

- **Entropy** (alternative to Gini Impurity):
  
  $$ Entropy = - \sum_{i=1}^{n} p_i \log_2(p_i) $$

- **Information Gain** (used to decide which feature to split on):

  $$ IG(D, f) = Entropy(D) - \sum_{v \in Values(f)} \frac{|D_v|}{|D|} Entropy(D_v) $$

  Where:
  - `D` is the dataset,
  - `f` is the feature to split on,
  - `Values(f)` is the set of all possible values for feature `f`,
  - `D_v` is the subset of `D` that has value `v` for feature `f`.

## Advantages

- It can handle thousands of input variables without variable deletion.
- It provides a higher level of accuracy.
- Runs efficiently on large datasets.
- It can handle missing values and maintains accuracy when a large proportion of the data are missing.

## Disadvantages

- Random Forest models are not all that interpretable; they are like black boxes.
- For very large data sets, the size of the trees can take up a lot of memory.
- While **Random Forest** is less prone to overfitting compared to individual decision trees, it is not entirely immune to this issue, especially in certain situations:

  - **With extremely noisy data**: If the data contains a lot of noise, Random Forest can still overfit by capturing the noise in the training data across its many trees, especially if the number of trees (`n_estimators`) is very high.

  - **Lack of tree diversity**: If the random subsets of features and data points used to train individual trees are not diverse enough, the ensemble might still overfit by being too tailored to the training data.

  - **Complex models with insufficient regularization**: Without proper tuning of the model's hyperparameters (such as `max_depth` for tree depth, `min_samples_split` for the minimum number of samples required to split an internal node, etc.), a Random Forest can grow overly complex trees, which may lead to overfitting.



In [10]:
%%writefile ../../src/models/random_forest.py
import numpy as np
from scipy.stats import mode
from src.models.cart import CART
from joblib import Parallel, delayed

class RandomForestClassifier:
    def __init__(self, n_estimators=100, max_features='sqrt', max_depth=None, 
                 min_samples_split=2, min_impurity_decrease=0, bootstrap=True):
        self.n_estimators = n_estimators  # Number of trees
        self.max_features = max_features  # The number of features to consider when looking for the best split
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_impurity_decrease = min_impurity_decrease
        self.bootstrap = bootstrap  # Whether bootstrap samples are used when building trees
        self.trees = []  # List to store all fitted tree models

    def fit(self, X, y):
        self.trees = []
        
        # Parallelize tree fitting
        self.trees = Parallel(n_jobs=-1)(delayed(self._fit_tree)(X, y) for _ in range(self.n_estimators))
        
            
    def _fit_tree(self, X, y):
        
        n_samples, n_features = X.shape
        # Bootstrap sampling
        if self.bootstrap:
            indices = np.random.choice(n_samples, size=n_samples, replace=True)
            X_sample, y_sample = X[indices], y[indices]
        else:
            X_sample, y_sample = X, y
        
        # Handle max_features
        if self.max_features == 'sqrt':
            max_features = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            max_features = int(np.log2(n_features))
        elif isinstance(self.max_features, float):
            max_features = int(self.max_features * n_features)
        elif isinstance(self.max_features, int):
            max_features = self.max_features
        else:
            max_features = n_features
        
        # Initialize and fit a tree model
        tree = CART(max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split,
                    min_impurity_decrease=self.min_impurity_decrease,
                    max_features=max_features)
        tree.fit(X_sample, y_sample)
        return tree

    def predict(self, X):
        # Collect predictions from each tree
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        
        # For classification, take the mode (most common label) across trees
        mode_preds, _ = mode(tree_preds, axis=0)
        return mode_preds


Writing ../../src/models/random_forest.py


In [ ]:
import numpy as np
from src.data.load_dataset import load_spambase
from src.models.random_forest import RandomForestClassifier

from sklearn.model_selection import train_test_split

In [7]:
X, y = load_spambase()
# Split the dataset into training+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Further split the training data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, stratify=y_train_val, random_state=42) # 0.25 x 0.8 = 0.2

X_train.shape, X_val.shape, X_test.shape

((2760, 57), (920, 57), (921, 57))

In [8]:
rf = RandomForestClassifier(n_estimators=200, max_features=42, max_depth=10, min_samples_split=5, min_impurity_decrease=0, bootstrap=True)
rf.fit(X_train, y_train)

In [11]:
y_pred = rf.predict(X_val)
accuracy = np.mean(y_pred == y_val)
print(f'Validation accuracy: {accuracy:.2f}')

Validation accuracy: 0.94
